In [1]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
to_fit = pd.read_csv("Data/qaly_input.csv")

In [3]:
to_fit

,Description,Value,Lower Bound,Upper Bound
0,Child-Ototoxicity,0.83,0.77,0.89
1,Child-NCD,0.67,0.56,0.74
2,Mother-Ototoxicity or NCD,0.85,0.74,0.94
3,Mother-Infant Death,0.92,0.60,0.99


In [4]:
# fits an encompassing beta distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_beta_distr(value, lower_bound, upper_bound):
    og_alpha = value
    og_beta = 1-og_alpha
    
    for scale in range(200):
        scale = 200 - scale
        alpha = og_alpha * scale
        beta = og_beta * scale
        ppf_025 = ss.beta.ppf(0.025, alpha, beta)
        ppf_975 = ss.beta.ppf(0.975, alpha, beta)
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            print("alpha: %.2f \t beta: %.2f" % (alpha, beta))           
            return alpha, beta
        
    print("No match found")
    return 0, 0

In [5]:
to_fit["alpha"], to_fit["beta"] = np.vectorize(fit_encompassing_beta_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

alpha: 104.58 	 beta: 21.42
alpha: 104.58 	 beta: 21.42
alpha: 49.58 	 beta: 24.42
alpha: 34.85 	 beta: 6.15
alpha: 4.60 	 beta: 0.40


In [6]:
to_fit["beta_mean"] = ss.beta.mean(to_fit["alpha"], to_fit["beta"])
to_fit["beta_2.5pct"] = ss.beta.ppf(0.025, to_fit["alpha"], to_fit["beta"])
to_fit["beta_97.5pct"] = ss.beta.ppf(0.975, to_fit["alpha"], to_fit["beta"])

In [7]:
to_fit

,Description,Value,Lower Bound,Upper Bound,alpha,beta,beta_mean,beta_2.5pct,beta_97.5pct
0,Child-Ototoxicity,0.83,0.77,0.89,104.58,21.42,0.83,0.759965,0.890127
1,Child-NCD,0.67,0.56,0.74,49.58,24.42,0.67,0.559606,0.771689
2,Mother-Ototoxicity or NCD,0.85,0.74,0.94,34.85,6.15,0.85,0.727356,0.940517
3,Mother-Infant Death,0.92,0.60,0.99,4.60,0.40,0.92,0.597570,0.999983


In [8]:
to_fit.to_csv("Data/qaly_output.csv")